In [17]:
import pyodbc
import pandas as pd
import numpy as np
import urllib
from sqlalchemy import create_engine
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.ensemble import RandomForestClassifier

# Conexion
conn_str = (
    "DRIVER={ODBC Driver 17 for SQL Server};"
    "SERVER=192.168.1.68;"
    "DATABASE=DM_HistorialAlumnosTI;"
    "Trusted_Connection=yes;"
)

conn_url = f"mssql+pyodbc:///?odbc_connect={urllib.parse.quote_plus(conn_str)}"
engine = create_engine(conn_url)
conn = engine.connect()

#Consulta
query = """
SELECT * FROM vw_ModeloPrediccion1
"""

df = pd.read_sql(query, engine)
conn.close()
print(df)

     Matricula  PromedioGeneral  PromedioMateriasRelacionadas  \
0      2000055              NaN                           NaN   
1      2000067              NaN                           NaN   
2      2000087              NaN                           NaN   
3      2001020              NaN                           NaN   
4      2001073              NaN                           NaN   
...        ...              ...                           ...   
1811     99056              NaN                           NaN   
1812     99091              NaN                           NaN   
1813     99132              NaN                           NaN   
1814     99142              NaN                           NaN   
1815     99144              NaN                           NaN   

      PorcentajeReprobacion  VecesReprobadas  TotalMateriasInscritas  \
0                 53.174603              NaN                     NaN   
1                 53.174603              NaN                     NaN   
2  

In [19]:
# Reemplazar valores nulos en VecesReprobadas y VecesExtraordinario con 0 (ya que indica que nunca ha reprobado)
df.fillna(0, inplace=True)
print(df)

     Matricula  PromedioGeneral  PromedioMateriasRelacionadas  \
0      2000055              0.0                           0.0   
1      2000067              0.0                           0.0   
2      2000087              0.0                           0.0   
3      2001020              0.0                           0.0   
4      2001073              0.0                           0.0   
...        ...              ...                           ...   
1811     99056              0.0                           0.0   
1812     99091              0.0                           0.0   
1813     99132              0.0                           0.0   
1814     99142              0.0                           0.0   
1815     99144              0.0                           0.0   

      PorcentajeReprobacion  VecesReprobadas  TotalMateriasInscritas  \
0                 53.174603              0.0                     0.0   
1                 53.174603              0.0                     0.0   
2  

In [21]:

# 3️⃣ Definir la variable objetivo (1 = Aprobará, 0 = No aprobará)
df["Aprobará"] = (df["PromedioMateriasRelacionadas"] >= 7).astype(int)

print(df)

     Matricula  PromedioGeneral  PromedioMateriasRelacionadas  \
0      2000055              0.0                           0.0   
1      2000067              0.0                           0.0   
2      2000087              0.0                           0.0   
3      2001020              0.0                           0.0   
4      2001073              0.0                           0.0   
...        ...              ...                           ...   
1811     99056              0.0                           0.0   
1812     99091              0.0                           0.0   
1813     99132              0.0                           0.0   
1814     99142              0.0                           0.0   
1815     99144              0.0                           0.0   

      PorcentajeReprobacion  VecesReprobadas  TotalMateriasInscritas  \
0                 53.174603              0.0                     0.0   
1                 53.174603              0.0                     0.0   
2  

In [23]:
# 4️⃣ Definir variables predictoras y objetivo
X = df.drop(columns=["Matricula", "Aprobará"])  # Excluir Matricula porque no es una variable predictora
y = df["Aprobará"]

print(f"Valores de X: \n{X}")
print(f"Valores en Y: \n{y}")

Valores de X: 
      PromedioGeneral  PromedioMateriasRelacionadas  PorcentajeReprobacion  \
0                 0.0                           0.0              53.174603   
1                 0.0                           0.0              53.174603   
2                 0.0                           0.0              53.174603   
3                 0.0                           0.0              53.174603   
4                 0.0                           0.0              53.174603   
...               ...                           ...                    ...   
1811              0.0                           0.0              53.174603   
1812              0.0                           0.0              53.174603   
1813              0.0                           0.0              53.174603   
1814              0.0                           0.0              53.174603   
1815              0.0                           0.0              53.174603   

      VecesReprobadas  TotalMateriasInscritas  V

In [25]:

# 5️⃣ Dividir datos en entrenamiento (80%) y prueba (20%)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(f"X_train: \n{X_train}")
print(f"y_train: \n{y_train}")

X_train: 
      PromedioGeneral  PromedioMateriasRelacionadas  PorcentajeReprobacion  \
292               0.0                           0.0              53.174603   
1416              6.0                           8.0              53.174603   
1547              0.0                           2.0              53.174603   
300               0.0                           0.0              53.174603   
1703              8.0                           7.0              53.174603   
...               ...                           ...                    ...   
1130              3.0                           4.0              53.174603   
1294              9.0                           9.0              53.174603   
860               9.0                           0.0              53.174603   
1459              6.0                           2.0              53.174603   
1126              8.0                           9.0              53.174603   

      VecesReprobadas  TotalMateriasInscritas  VecesE

In [27]:
# 6️⃣ Inicializar y entrenar el modelo Random Forest
modelo = RandomForestClassifier(n_estimators=100, random_state=42)
modelo.fit(X_train, y_train)

RandomForestClassifier(random_state=42)

In [29]:
# 7️⃣ Hacer predicciones
y_pred = modelo.predict(X_test)
# 8️⃣ Evaluar el modelo
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Reporte de Clasificación:\n", classification_report(y_test, y_pred))

Accuracy: 1.0
Reporte de Clasificación:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00       248
           1       1.00      1.00      1.00       116

    accuracy                           1.00       364
   macro avg       1.00      1.00      1.00       364
weighted avg       1.00      1.00      1.00       364



In [78]:
import pandas as pd
import numpy as np

def predecir_aprobacion_como_porcentaje(promedio_general, promedio_materias_relacionadas, porcentaje_reprobacion, veces_reprobadas, total_materias_inscritas, veces_extraordinario):
    # Creamos un DataFrame con las características del estudiante, asegurándonos de tener las mismas columnas que usó el modelo
    datos_estudiante = pd.DataFrame({
        'PromedioGeneral': [promedio_general],
        'PromedioMateriasRelacionadas': [promedio_materias_relacionadas],
        'PorcentajeReprobacion': [porcentaje_reprobacion],
        'VecesReprobadas': [veces_reprobadas],
        'TotalMateriasInscritas': [total_materias_inscritas],
        'VecesExtraordinario': [veces_extraordinario]
    })
    
    # Hacer la predicción de la probabilidad (predict_proba devuelve probabilidades para cada clase)
    probabilidad = modelo.predict_proba(datos_estudiante)
    
    # probabilidad[0, 1] corresponde a la probabilidad de que el alumno apruebe (1)
    probabilidad_aprobacion = probabilidad[0, 1] * 100  # Convertimos a porcentaje
    
    return f"El estudiante tiene un {probabilidad_aprobacion:.2f}% de probabilidad de aprobar."

# Datos del estudiante
promedio_general = 8
promedio_materias_relacionadas = 6.4
porcentaje_reprobacion = 90.0
veces_reprobadas = 1
total_materias_inscritas = 6
veces_extraordinario = 0

# Predecir
resultado = predecir_aprobacion_como_porcentaje(promedio_general, promedio_materias_relacionadas, porcentaje_reprobacion, veces_reprobadas, total_materias_inscritas, veces_extraordinario)
print(resultado)


El estudiante tiene un 7.00% de probabilidad de aprobar.
